# **ANN + FINBERT**

In [ ]:
# -------------------------------
# Step 0: Imports & Setup
# -------------------------------
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.utils import resample
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Reproducibility
np.random.seed(42)
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🖥️ Using device:", device)

nltk.download('stopwords')
nltk.download('wordnet')

# -------------------------------
# Step 1: Load Dataset
# -------------------------------
df = pd.read_csv(r"C:\Users\VARSHINI.M\OneDrive\Desktop\7th sem\interdisclipinary\Datasets\Sentiment Analysis for Financial News\all-data.csv", encoding='latin-1', header=None)
df.columns = ['sentiment', 'text']

sentiment_map = {'negative': -1, 'neutral': 0, 'positive': 1}
df['sentiment'] = df['sentiment'].map(sentiment_map)
df.dropna(subset=['sentiment'], inplace=True)
df['sentiment'] = df['sentiment'].astype(int)
label_text_map = {-1: "Negative", 0: "Neutral", 1: "Positive"}

print("📊 Original Class Distribution:")
print(df['sentiment'].map(label_text_map).value_counts(), "\n")

# -------------------------------
# Step 1.5: Handle Imbalance
# -------------------------------
df_neg = df[df['sentiment'] == -1]
df_neu = df[df['sentiment'] == 0]
df_pos = df[df['sentiment'] == 1]
max_size = max(len(df_neg), len(df_neu), len(df_pos))

df_balanced = pd.concat([
    resample(df_neg, replace=True, n_samples=max_size, random_state=42),
    resample(df_neu, replace=True, n_samples=max_size, random_state=42),
    resample(df_pos, replace=True, n_samples=max_size, random_state=42)
]).sample(frac=1, random_state=42).reset_index(drop=True)

print("✅ Balanced Class Distribution:")
print(df_balanced['sentiment'].map(label_text_map).value_counts(), "\n")

# -------------------------------
# Step 2: Text Preprocessing
# -------------------------------
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
    return " ".join(tokens)

df_balanced['clean_text'] = df_balanced['text'].astype(str).apply(clean_text)

# -------------------------------
# Step 3: Data Split
# -------------------------------
X_train, X_temp, y_train, y_temp = train_test_split(
    df_balanced['clean_text'], df_balanced['sentiment'],
    test_size=0.3, stratify=df_balanced['sentiment'], random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)
print(f"📚 Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

# -------------------------------
# Step 4: Tokenization
# -------------------------------
tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")

def encode_texts(texts, max_len=128):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )

train_enc = encode_texts(X_train)
val_enc = encode_texts(X_val)
test_enc = encode_texts(X_test)

sentiment_to_idx = {-1: 0, 0: 1, 1: 2}
idx_to_label = {0: "Negative", 1: "Neutral", 2: "Positive"}

y_train_idx = y_train.map(sentiment_to_idx)
y_val_idx = y_val.map(sentiment_to_idx)
y_test_idx = y_test.map(sentiment_to_idx)

# -------------------------------
# Step 5: Dataset Class
# -------------------------------
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

train_ds = SentimentDataset(train_enc, torch.tensor(y_train_idx.values))
val_ds = SentimentDataset(val_enc, torch.tensor(y_val_idx.values))
test_ds = SentimentDataset(test_enc, torch.tensor(y_test_idx.values))

# -------------------------------
# Step 6: FinBERT + ANN Model
# -------------------------------
class FinBERT_ANN(nn.Module):
    def __init__(self, hidden_dim=256, num_classes=3):
        super(FinBERT_ANN, self).__init__()
        self.bert = BertModel.from_pretrained("ProsusAI/finbert")
        self.fc1 = nn.Linear(self.bert.config.hidden_size, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, num_classes)
    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output.pooler_output
        x = self.fc1(pooled_output)
        x = self.relu(x)
        x = self.dropout(x)
        return self.fc2(x)

model = FinBERT_ANN().to(device)

# -------------------------------
# Step 7: Training
# -------------------------------
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)

train_accs, val_accs, test_accs = [], [], []
train_losses, val_losses, test_losses = [], [], []

def compute_accuracy_and_loss(loader):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            ids = batch['input_ids'].to(device)
            mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(ids, mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
    return acc, total_loss / len(loader), np.array(all_labels), np.array(all_preds)

EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    total_train_loss, correct_train, total_train = 0, 0, 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        optimizer.zero_grad()
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(ids, mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
    train_acc = correct_train / total_train
    train_loss = total_train_loss / len(train_loader)
    val_acc, val_loss, _, _ = compute_accuracy_and_loss(val_loader)
    test_acc, test_loss, _, _ = compute_accuracy_and_loss(test_loader)
    train_accs.append(train_acc); val_accs.append(val_acc); test_accs.append(test_acc)
    train_losses.append(train_loss); val_losses.append(val_loss); test_losses.append(test_loss)
    print(f"\n📘 EPOCH {epoch+1}/{EPOCHS} SUMMARY")
    print(f"Train→ Acc: {train_acc*100:.2f}% | Loss: {train_loss:.4f}")
    print(f"Val  → Acc: {val_acc*100:.2f}% | Loss: {val_loss:.4f}")
    print(f"Test → Acc: {test_acc*100:.2f}% | Loss: {test_loss:.4f}")
    print("-"*60)

# -------------------------------
# Step 8: Visualization
# -------------------------------
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS), train_losses, label="Train Loss")
plt.plot(range(EPOCHS), val_losses, label="Val Loss")
plt.plot(range(EPOCHS), test_losses, label="Test Loss")
plt.title("Loss vs Epochs"); plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()

plt.subplot(1,2,2)
plt.plot(range(EPOCHS), train_accs, label="Train Acc")
plt.plot(range(EPOCHS), val_accs, label="Val Acc")
plt.plot(range(EPOCHS), test_accs, label="Test Acc")
plt.title("Accuracy vs Epochs"); plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend()
plt.tight_layout()
plt.show()

# -------------------------------
# Step 9: Confusion Matrices
# -------------------------------
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(cm, display_labels=["Negative", "Neutral", "Positive"])
    disp.plot(cmap="Blues")
    plt.title(title)
    plt.show()

train_acc, _, y_train_true, y_train_pred = compute_accuracy_and_loss(train_loader)
val_acc, _, y_val_true, y_val_pred = compute_accuracy_and_loss(val_loader)
test_acc, _, y_test_true, y_test_pred = compute_accuracy_and_loss(test_loader)

plot_confusion_matrix(y_train_true, y_train_pred, "Train Confusion Matrix")
plot_confusion_matrix(y_val_true, y_val_pred, "Validation Confusion Matrix")
plot_confusion_matrix(y_test_true, y_test_pred, "Test Confusion Matrix")

# Overall Confusion Matrix
y_all_true = np.concatenate([y_train_true, y_val_true, y_test_true])
y_all_pred = np.concatenate([y_train_pred, y_val_pred, y_test_pred])
plot_confusion_matrix(y_all_true, y_all_pred, "Overall Confusion Matrix")

# -------------------------------
# Step 10: Evaluation Report
# -------------------------------
print("\n📊 FINAL TEST REPORT:")
print(classification_report(y_test_true, y_test_pred, target_names=["Negative", "Neutral", "Positive"]))
final_acc = accuracy_score(y_test_true, y_test_pred)
print(f"✅ Final Test Accuracy: {final_acc*100:.2f}%")

# -------------------------------
# Step 11: Save Model
# -------------------------------
torch.save(model.state_dict(), "finbert_ann_balanced_model.pth")
print("✅ Model saved as 'finbert_ann_balanced_model.pth'")

# -------------------------------
# Step 12: Real-Time Prediction
# -------------------------------
def predict_sentiment(sentence):
    model.eval()
    cleaned = clean_text(sentence)
    encoded = tokenizer(cleaned, return_tensors='pt', truncation=True, padding=True, max_length=128)
    input_ids = encoded['input_ids'].to(device)
    attn_mask = encoded['attention_mask'].to(device)
    with torch.no_grad():
        logits = model(input_ids, attn_mask)
        pred = torch.argmax(logits, dim=1).item()
    label = idx_to_label[pred]
    return f"{label} ({['-1','0','1'][pred]})"

print("\n💬 Example Predictions:")
examples = [
    "The company reported record profits this quarter!",
    "The stock market is unstable due to new regulations.",
    "Investors are unsure about the future of this firm."
]
for s in examples:
    print(f"{s} → {predict_sentiment(s)}")


🖥️ Using device: cpu
📊 Original Class Distribution:
sentiment
Neutral     2879
Positive    1363
Negative     604
Name: count, dtype: int64 

✅ Balanced Class Distribution:
sentiment
Negative    2879
Positive    2879
Neutral     2879
Name: count, dtype: int64 



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VARSHINI.M\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VARSHINI.M\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


📚 Train: 6045 | Val: 1296 | Test: 1296


Epoch 1/10: 100%|████████████████████████████████████████████████████████████████████| 378/378 [09:31<00:00,  1.51s/it]



📘 EPOCH 1/10 SUMMARY
Train→ Acc: 69.98% | Loss: 0.7276
Val  → Acc: 75.39% | Loss: 0.6025
Test → Acc: 76.70% | Loss: 0.6071
------------------------------------------------------------


Epoch 2/10: 100%|████████████████████████████████████████████████████████████████████| 378/378 [10:21<00:00,  1.64s/it]



📘 EPOCH 2/10 SUMMARY
Train→ Acc: 72.80% | Loss: 0.6540
Val  → Acc: 76.39% | Loss: 0.5941
Test → Acc: 77.16% | Loss: 0.5944
------------------------------------------------------------


Epoch 3/10:  35%|████████████████████████                                            | 134/378 [02:55<05:11,  1.28s/it]

# **CNN + FINBERT**

In [ ]:
# -------------------------------
# Step 0: Imports & Setup
# -------------------------------
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.utils import resample
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Reproducibility
np.random.seed(42)
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🖥️ Using device:", device)

nltk.download('stopwords')
nltk.download('wordnet')

# -------------------------------
# Step 1: Load Dataset
# -------------------------------
df = pd.read_csv("/content/all-data.csv", encoding='latin-1', header=None)
df.columns = ['sentiment', 'text']

sentiment_map = {'negative': -1, 'neutral': 0, 'positive': 1}
df['sentiment'] = df['sentiment'].map(sentiment_map)
df.dropna(subset=['sentiment'], inplace=True)
df['sentiment'] = df['sentiment'].astype(int)
label_text_map = {-1: "Negative", 0: "Neutral", 1: "Positive"}

print("📊 Original Class Distribution:")
print(df['sentiment'].map(label_text_map).value_counts(), "\n")

# -------------------------------
# Step 1.5: Handle Imbalance
# -------------------------------
df_neg = df[df['sentiment'] == -1]
df_neu = df[df['sentiment'] == 0]
df_pos = df[df['sentiment'] == 1]

max_size = max(len(df_neg), len(df_neu), len(df_pos))
df_neg_over = resample(df_neg, replace=True, n_samples=max_size, random_state=42)
df_neu_over = resample(df_neu, replace=True, n_samples=max_size, random_state=42)
df_pos_over = resample(df_pos, replace=True, n_samples=max_size, random_state=42)

df_balanced = pd.concat([df_neg_over, df_neu_over, df_pos_over])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print("✅ Balanced Class Distribution:")
print(df_balanced['sentiment'].map(label_text_map).value_counts(), "\n")

# -------------------------------
# Step 2: Text Preprocessing
# -------------------------------
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
    return " ".join(tokens)

df_balanced['clean_text'] = df_balanced['text'].astype(str).apply(clean_text)

# -------------------------------
# Step 3: Data Split
# -------------------------------
X_train, X_temp, y_train, y_temp = train_test_split(
    df_balanced['clean_text'], df_balanced['sentiment'],
    test_size=0.3, stratify=df_balanced['sentiment'], random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

print(f"📚 Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

# -------------------------------
# Step 4: Tokenization
# -------------------------------
tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")

def encode_texts(texts, max_len=128):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )

train_enc = encode_texts(X_train)
val_enc = encode_texts(X_val)
test_enc = encode_texts(X_test)

sentiment_to_idx = {-1: 0, 0: 1, 1: 2}
idx_to_sentiment = {0: -1, 1: 0, 2: 1}
idx_to_label = {0: "Negative", 1: "Neutral", 2: "Positive"}

y_train_idx = y_train.map(sentiment_to_idx)
y_val_idx = y_val.map(sentiment_to_idx)
y_test_idx = y_test.map(sentiment_to_idx)

# -------------------------------
# Step 5: Dataset Class
# -------------------------------
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

train_ds = SentimentDataset(train_enc, torch.tensor(y_train_idx.values))
val_ds = SentimentDataset(val_enc, torch.tensor(y_val_idx.values))
test_ds = SentimentDataset(test_enc, torch.tensor(y_test_idx.values))

# -------------------------------
# Step 6: FinBERT + CNN MODEL
# -------------------------------
class FinBERT_CNN(nn.Module):
    def __init__(self, num_classes=3, num_filters=128, filter_sizes=[2, 3, 4]):
        super(FinBERT_CNN, self).__init__()
        self.bert = BertModel.from_pretrained("ProsusAI/finbert")
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=self.bert.config.hidden_size,
                      out_channels=num_filters,
                      kernel_size=k)
            for k in filter_sizes
        ])
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(num_filters * len(filter_sizes), num_classes)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # Freeze FinBERT
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = bert_output.last_hidden_state.transpose(1, 2)
        conv_results = [torch.relu(conv(x)) for conv in self.convs]
        pooled = [torch.max(c, dim=2)[0] for c in conv_results]
        cat = torch.cat(pooled, dim=1)
        out = self.dropout(cat)
        return self.fc(out)

model = FinBERT_CNN().to(device)

# -------------------------------
# Step 7: Training
# -------------------------------
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)

train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

def compute_accuracy_and_loss(loader):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    preds_all, labels_all = [], []
    with torch.no_grad():
        for batch in loader:
            ids = batch['input_ids'].to(device)
            mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(ids, mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            preds_all.extend(preds.cpu().numpy())
            labels_all.extend(labels.cpu().numpy())
    return correct / total, total_loss / len(loader), preds_all, labels_all

EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    total_train_loss, correct_train, total_train = 0, 0, 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        optimizer.zero_grad()
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(ids, mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_acc = correct_train / total_train
    train_loss = total_train_loss / len(train_loader)
    val_acc, val_loss, _, _ = compute_accuracy_and_loss(val_loader)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"\n📘 EPOCH {epoch+1}/{EPOCHS} SUMMARY")
    print(f"Train  → Acc: {train_acc*100:.2f}% | Loss: {train_loss:.4f}")
    print(f"Val    → Acc: {val_acc*100:.2f}% | Loss: {val_loss:.4f}")
    print("-"*60)

# -------------------------------
# Step 8: Accuracy & Loss Plots
# -------------------------------
plt.figure(figsize=(10,5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("Loss vs Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
plt.plot(train_accuracies, label="Train Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.title("Accuracy vs Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# -------------------------------
# Step 9: Confusion Matrices
# -------------------------------
def plot_conf_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=["Negative", "Neutral", "Positive"],
                yticklabels=["Negative", "Neutral", "Positive"])
    plt.title(title)
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.show()

train_acc, _, y_pred_train, y_true_train = compute_accuracy_and_loss(train_loader)
val_acc, _, y_pred_val, y_true_val = compute_accuracy_and_loss(val_loader)
test_acc, _, y_pred_test, y_true_test = compute_accuracy_and_loss(test_loader)

print("\n📊 Confusion Matrices:")
plot_conf_matrix(y_true_train, y_pred_train, "Train Confusion Matrix")
plot_conf_matrix(y_true_val, y_pred_val, "Validation Confusion Matrix")
plot_conf_matrix(y_true_test, y_pred_test, "Test Confusion Matrix")

# Overall CM
y_true_all = y_true_train + y_true_val + y_true_test
y_pred_all = y_pred_train + y_pred_val + y_pred_test
plot_conf_matrix(y_true_all, y_pred_all, "Overall Confusion Matrix")

# -------------------------------
# Step 10: Final Evaluation
# -------------------------------
def evaluate_model(model, dataset, y_true_idx):
    loader = DataLoader(dataset, batch_size=32)
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in loader:
            ids = batch['input_ids'].to(device)
            mask = batch['attention_mask'].to(device)
            outputs = model(ids, mask)
            preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
    y_true_text = [idx_to_label[i] for i in y_true_idx]
    y_pred_text = [idx_to_label[i] for i in preds]
    print("\n📊 FINAL TEST REPORT:")
    print(classification_report(y_true_text, y_pred_text))
    return accuracy_score(y_true_idx, preds)

final_acc = evaluate_model(model, test_ds, y_test_idx)
print(f"✅ Final Test Accuracy: {final_acc*100:.2f}%")

# -------------------------------
# Step 11: Save Model
# -------------------------------
torch.save(model.state_dict(), "finbert_cnn_balanced_model.pth")
print("✅ Model saved as 'finbert_cnn_balanced_model.pth'")

# -------------------------------
# Step 12: Real-Time Prediction
# -------------------------------
def predict_sentiment(sentence):
    model.eval()
    cleaned = clean_text(sentence)
    encoded = tokenizer(cleaned, return_tensors='pt', truncation=True, padding=True, max_length=128)
    input_ids = encoded['input_ids'].to(device)
    attn_mask = encoded['attention_mask'].to(device)
    with torch.no_grad():
        logits = model(input_ids, attn_mask)
        pred = torch.argmax(logits, dim=1).item()
    sentiment_val = idx_to_sentiment[pred]
    return f"{idx_to_label[pred]} ({sentiment_val})"

print("\n💬 Example Predictions:")
examples = [
    "The company reported record profits this quarter!",
    "The stock market is unstable due to new regulations.",
    "Investors are unsure about the future of this firm."
]
for s in examples:
    print(f"{s} → {predict_sentiment(s)}")
